In [5]:
import requests
from time import sleep
import random
import json
import yaml
import sqlalchemy
from sqlalchemy import text
import pandas as pd
from datetime import datetime

random.seed(100)

In [8]:
class AWSDBConnector:

    def __init__(self):
        
        self.database_credentials = None
        self.engine = None
    
    
    def read_db_creds(self, credential_file):
        with open(credential_file, 'r') as file:
            self.database_credentials = yaml.safe_load(file)

    
    def init_db_engine(self):
        self.engine = sqlalchemy.create_engine(
            f"mysql+pymysql://{self.database_credentials['USER']}"
            f":{self.database_credentials['PASSWORD']}"
            f"@{self.database_credentials['HOST']}"
            f":{self.database_credentials['PORT']}"
            f"/{self.database_credentials['DATABASE']}?charset=utf8mb4"
        )

new_connector = AWSDBConnector()
new_connector.read_db_creds('Sensitive data/db_credentials.yaml')
new_connector.init_db_engine()


In [9]:
def function(sql_querry):
        with new_connector.engine.connect() as connection:
            pin_string = text(sql_querry)
            pin_result = connection.execute(pin_string)
            pin_db = pd.DataFrame(pin_result)
            return pin_db.loc[0].to_json(date_format='iso')

In [10]:
def post_api(topic, payload):
    invoke_url = "https://m1c8pv5ln1.execute-api.us-east-1.amazonaws.com/test/topics/"
    #headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
    request = requests.post(f"{invoke_url}{topic}", data=payload)
    print(request.status_code)

In [11]:
def run():
#     while True:
        sleep(random.randrange(0, 2))
        random_row = random.randint(0, 11000)

        with new_connector.engine.connect() as connection:
            pin_result = function(f"SELECT * FROM pinterest_data LIMIT {random_row}, 1")
            geo_result = function(f"SELECT * FROM geolocation_data LIMIT {random_row}, 1")
            user_result = function(f"SELECT * FROM user_data LIMIT {random_row}, 1")

            # post_api('0aa58e5ad07d.pin', pin_result)
            # post_api('0aa58e5ad07d.geo', geo_result)
            # post_api('0aa58e5ad07d.user', user_result)

            print(pin_result)
            print(geo_result)
            print(user_result)

            for item in pin_result:
                 print(type(pin_result[item]))

            print(type(pin_result))
            print(type(geo_result))
            print(type(user_result))

In [12]:
run()

{"index":7528,"unique_id":"fbe53c66-3442-4773-b19e-d3ec6f54dddf","title":"No Title Data Available","description":"No description available Story format","poster_name":"User Info Error","follower_count":"User Info Error","tag_list":"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e","is_image_or_video":"multi-video(story page format)","image_src":"Image src error.","downloaded":0,"save_location":"Local save in \/data\/mens-fashion","category":"mens-fashion"}
{"ind":7528,"timestamp":"2020-08-28T03:52:47.000","latitude":-89.9787,"longitude":-173.293,"country":"Albania"}
{"ind":7528,"first_name":"Abigail","last_name":"Ali","age":20,"date_joined":"2015-10-24T11:23:51.000"}


TypeError: string indices must be integers, not 'str'

In [13]:
def run_infinite_post_data_loop():
    #while True:
        sleep(random.randrange(0, 2))
        random_row = random.randint(0, 11000)

        with new_connector.engine.connect() as connection:

            pin_string = text(f"SELECT * FROM pinterest_data LIMIT {random_row}, 1")
            pin_selected_row = connection.execute(pin_string)
            
            for row in pin_selected_row:
                pin_result = row._mapping

            geo_string = text(f"SELECT * FROM geolocation_data LIMIT {random_row}, 1")
            geo_selected_row = connection.execute(geo_string)
            
            for row in geo_selected_row:
                geo_result = dict(row._mapping)

            user_string = text(f"SELECT * FROM user_data LIMIT {random_row}, 1")
            user_selected_row = connection.execute(user_string)
            
            for row in user_selected_row:
                user_result = dict(row._mapping)
            
           # print(pin_result)
            print(geo_result)
            #print(user_result)

            #print(type(pin_result))
            print(type(geo_result))
            #print(type(user_result))

            for item in geo_result:
                print(type(geo_result[item]))
                if type(geo_result[item]) == datetime:
                    print(geo_result[item])
                    geo_result[item] = str(geo_result[item])
                    print(geo_result[item])
                    print(type(geo_result[item]))
                    print(type(geo_result))
            
            for item in pin_result:
                 print(type(pin_result[item]))
            for item in user_result:
                 print(type(user_result[item]))
            



In [14]:
run_infinite_post_data_loop()

{'ind': 2863, 'timestamp': datetime.datetime(2020, 4, 27, 13, 34, 16), 'latitude': -5.34445, 'longitude': -177.924, 'country': 'Armenia'}
<class 'dict'>
<class 'int'>
<class 'datetime.datetime'>
2020-04-27 13:34:16
2020-04-27 13:34:16
<class 'str'>
<class 'dict'>
<class 'float'>
<class 'float'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'datetime.datetime'>


In [16]:
with new_connector.engine.connect() as connection:
    pin_string = text(f"SELECT * FROM pinterest_data LIMIT 5, 1")
    pin_selected_row = connection.execute(pin_string)
    pd_pin = pd.DataFrame(pin_selected_row)

In [21]:
pd_pin.head()

,index,unique_id,title,description,poster_name,follower_count,tag_list,is_image_or_video,image_src,downloaded,save_location,category
0,5,08fabbb2-8b80-41e8-871c-759837ed816f,Bulgarian Artist Makes Incredible Illustration...,"It doesn't matter if you use a pencil, a crayo...",Bored Panda,2M,"Girl Drawing Sketches,Pencil Art Drawings,Cool...",image,https://i.pinimg.com/originals/5f/38/b0/5f38b0...,1,Local save in /data/art,art


In [32]:
invoke_url = "https://m1c8pv5ln1.execute-api.us-east-1.amazonaws.com/test/topics/0aa58e5ad07d.user"
payload = pin_json
headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
response = requests.request("POST", invoke_url, headers=headers, data=payload)

print(response.status_code)
print(response.json())


422
{'error_code': 422, 'message': 'Unrecognized field: unique_id'}


In [3]:
test_dict = {'index': 2386, 'unique_id': '38c3e021-527e-4da6-8d43-c0aa07500d12', 'title': 'Christmas Five Senses Book', 'description': 'The sights, sounds and smells of Christmas stay with many of us for a lifetime. Perhaps cinnamon in your nasal cavity automatically takes you back to Grandma’s kitchen. Or at th…\xa0', 'poster_name': "Life Over C's", 'follower_count': '79k', 'tag_list': 'Christmas Books,Christmas Themes,Kids Christmas,Xmas,Christmas Projects,Preschool Christmas Crafts,Christmas Activities,Kindergarten Christmas,Winter Activities', 'is_image_or_video': 'image', 'image_src': 'https://i.pinimg.com/originals/3f/f7/54/3ff754d879c31507d1dc78104af06a63.jpg', 'downloaded': 1, 'save_location': 'Local save in /data/christmas', 'category': 'christmas'}

In [6]:
test_json = json.dumps(test_dict)

In [7]:
print(test_json)
print(type(test_json))

{"index": 2386, "unique_id": "38c3e021-527e-4da6-8d43-c0aa07500d12", "title": "Christmas Five Senses Book", "description": "The sights, sounds and smells of Christmas stay with many of us for a lifetime. Perhaps cinnamon in your nasal cavity automatically takes you back to Grandma\u2019s kitchen. Or at th\u2026\u00a0", "poster_name": "Life Over C's", "follower_count": "79k", "tag_list": "Christmas Books,Christmas Themes,Kids Christmas,Xmas,Christmas Projects,Preschool Christmas Crafts,Christmas Activities,Kindergarten Christmas,Winter Activities", "is_image_or_video": "image", "image_src": "https://i.pinimg.com/originals/3f/f7/54/3ff754d879c31507d1dc78104af06a63.jpg", "downloaded": 1, "save_location": "Local save in /data/christmas", "category": "christmas"}
<class 'str'>
